# Projeto 2: Análise Exploratória do Email Pessoal

*Baseado no código do capítulo 3 do livro "Hands on Exploratory Data Analysis with Python", com os códigos disponíveis no [github](https://github.com/PacktPublishing/Hands-on-Exploratory-Data-Analysis-with-Python)*.

#### Objetivo: extrair e pre-processar dados do email pessoal do gmail, para responder perguntas sobre os dados obtidos

#### __Construção do dataset__

__Etapa 1: Seleção dos itens a serem baixados__

Inicialmente nós temos que construir o conjunto de dados com o qual iremos trabalhar. Os dados aqui são oriundos do email pessoal de cada um, então por motivos mais que óbvios, espera-se que a probabilidade de duas contas terem os mesmos emails seja nula...

Para baixar os dados de seu email, considere os passos a seguir:

1. entre na sua conta gmail;
2. vá para o link: [https://takeout.google.com/](https://takeout.google.com/). Esse link o leva para uma página onde você pode escolher fazer download de qualquer serviço do google vinculado a sua conta. Desmarque todos, e, depois, encontre somente a opção E-mail e marque, como ilustra a figura abaixo:

![](opcoes_google_1.png)

3. Na opção "Todos os dados de email inclusos", você tem a opção de escolher que informações você quer que conste no dataset: caixa de entrada, lixeira, enviados, rascunho, etc. Sugiro escolher somente uma de forma a ter um conjunto de dados de bom tamanho, mas não gigantesco e que ajude você no exercícios de análise.

4. Após, isso, clique em "Próxima etapa" (final da página).

__Etapa 2: Escolha do tipo de arquivo, frequência e destino__

Nessa etapa você vai selecionar onde quer salvar os dados selecionados para download na etapa anterior. Seguiremos os seguintes passos:

1. Seleção do método de envio: a depender da quantidade de email que você espera baixar, minha sugestão é escolher o google drive, para evitar problemas de espaço;

2. Na frequência, escolha "exportar uma vez", para evitar que constantemente seja feito esse download (a menos que você deseje...)

3. Por fim, selecione o tipo de arquivo (.zip ou .tgz) e o tamanho mínimo do arquivo (se ficar maior, o arquivo será partido em parcelas do tamanho especificado por você).

Pronto!

Agora é só esperar a exportação terminar! 

___ATENÇÃO!!___ __esse processo pode demorar bastante, a depender da quantidade de dados que terá de ser exportado. Contabilize isso quando for começar o projeto para não haver atrasos!__

#### __Pré-processando o conjunto de dados__

Vamos começar carregando as bibliotecas que nós comumente usamos.

In [1]:
import pandas as pd
import mailbox

A biblioteca `mailbox` permite acessar e manipular diferentes tipos de formatação de emails e outros dados oriundos de mensagens na internet e nos será útil para processarmos os dados do arquivo `mbox` que foi baixado. Para mais informações, veja a [documentação](https://docs.python.org/pt-br/3/library/mailbox.html).

Para carregar os dados, basta fazer

In [2]:
mbox = mailbox.mbox('Categoria_ promoções.mbox') # emails.mbox é o nome que eu dei ao meu arquivo...
mbox

O arquivo com formato .mbox lembra um dicionário. Dessa forma, podemos verificar as chaves existentes nesse dicionário fazendo

In [3]:
for chaves in mbox[0]: print(chaves)

X-GM-THRID
X-Gmail-Labels
Delivered-To
Received
X-Google-Smtp-Source
X-Received
ARC-Seal
ARC-Message-Signature
ARC-Authentication-Results
Return-Path
Received
Received-SPF
Authentication-Results
DKIM-Signature
Mime-Version
Date
List-Unsubscribe
Message-ID
X-Campaign-ID
X-Email-Type
X-Email-ID
To
Reply-To
Feedback-ID
X-Mail-IP
From
Subject
Sender
Content-Type
Content-Transfer-Encoding


Cada chave dessa está relacionada a uma variável armazenada no conjunto de dados. Embora haja muitos objetos retornados pelos dados extraídos, não precisamos de todos os itens. Vamos extrair apenas os campos obrigatórios. A limpeza de dados é uma das etapas essenciais na fase de análise de dados. Para nossa análise, tudo que precisamos são dados para o seguinte: _subject_, _from_, _date_, _to_, _label_, e _thread_. 

Para fazer essa limpeza, vamos criar um um arquivo csv com adaptações para que consigamos ler o arquivo em um dataframe e realizar nossos trabalhos.

In [4]:
import csv

with open('emails.csv', 'w') as outputfile:
  writer = csv.writer(outputfile)
  writer.writerow(['subject','from','date','to',
                   'label','thread'])
    
  for message in mbox:
    writer.writerow([message['subject'], message['from'],  
                     message['date'], message['to'],  
                     message['X-Gmail-Labels'], message['X-GM-THRID']])

E agora podemos abrir o arquivo csv em um dataframe Pandas, contendo somente os campos que nos interessam.

In [5]:
df = pd.read_csv('emailss.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'emailss.csv'

In [ ]:
df

Vamos analisar nosso dataframe.

In [ ]:
df.info()

Notemos que a variável `date` está assinalada como `object`, mas é uma data, e deveria ser do tipo `datetime`, que é como o Pandas aloca variáveis relacionadas a datas.

Para fazer essa conversão, vamos usar a função `to_datetime` do Pandas, da seguinte forma:

In [ ]:
df['date'] = df['date'].apply(lambda x: pd.to_datetime(x, 
                                                       errors='coerce', 
                                                       utc=True))

E fazendo uma nova inspeção, vemos

In [ ]:
df.info()

Por último, precisamos fazer alguns refatoramentos. Por exemplo, ao inspecionar a variável `from`, obtemos 

In [ ]:
df['from']

e percebemos que não são alocados somente os emails de origem, mas algumas informações que não são necessariamente úteis para nós.

O processo de "limpar" essa informação extra é chamada de refatoramento, e, para nosso caso, usaremos uma abordagem baseada em expressões regulares para essa tarefa, usando a biblioteca `re` do Python (se você não sabe o que é uma expressão regular, dá uma lida nesse [tutorial](https://realpython.com/regex-python/) e divirta-se!).

In [ ]:
import re

Vamos criar uma função que pega um string em qualquer coluna e extrai somente a parte do email existente nela:

In [ ]:
def extracao_email(string):
  email = re.findall(r'<(.+?)>', string) #aqui está a expressão regular
  if not email:
    email = list(filter(lambda y: '@' in y, string.split()))
  return email[0] if email else np.nan #se tiver email, ok; senão, retorna nan.

Sugiro, caso não conheça todas as partes envolvidas nessa função, tentar buscar entender. Dá um belo exercício.

Agora, basta aplicar a função a coluna `from`.

In [ ]:
df['from'] = df['from'].apply(lambda x: extracao_email(x))

Inspecionando novamente, obtemos somente emails!

In [ ]:
df['from']

Vamos agora olhar para a variável `label`, que, como está mostrando, indica basicamente a "posição" do email na caixa de entrada: se foi um email recebido, se está na pasta de enviados, etc.

Paara facilitar nossa vida vamos modificar os valores apresentados no `label` para que tenhamos dois tipos de emails: "enviados" e "inbox". A lógica é bem simples: se o valor na variável `from` for o seu email pessoal, então é um email enviado, caso contrário, é inbox.

In [ ]:
df['label'] = df['from'].apply(lambda x: 
                               'enviado' if x=='viniapnm@gmail.com' else 'inbox')

In [ ]:
df['label'].unique()

Nossa última tarefa é tentar resolver os eventuais problemas relacionados a fuso horário. Para isso, precisamos ajustar os horários para uma _timezone_ relacionada a nossa localização, usando duas bibliotecas Python: `datetime` e `pytz`. Essa parte do processamento é baseada, também, em parte, nesse [tutorial](https://www.alura.com.br/artigos/lidando-com-datas-e-horarios-no-python) e na [documentação do pytz](https://pypi.org/project/pytz/).

In [ ]:
import datetime
import pytz

Precisamos, primeiramente, saber qual é a nossa _timezone_. Para isso, vamos fazer

In [ ]:
for tz in pytz.all_timezones: print(tz)

Observamos que o mais próximo da gente seria a opção `America/Recife`, por motivos óbvios.

Precisamos agora converter os horários para essa _timezone_, e, para isso, usaremos uma função

In [ ]:
def ref_timezone(x): return x.astimezone(pytz.timezone('America/Recife'))

E vamos usar essa função para ajustar as datas, na coluna `date`

In [ ]:
df['date'] = df['date'].apply(lambda x: ref_timezone(x))

Aqui temos um probleminha. Quando fizemos a conversão do tipo `object` para `datetime`, lá atrás, o próprio Pandas substitui as datas que possuem algum tipo de erro pelo tipo `NaT`, algo como um `nan` só que para datas. Nesses pontos em que a data é tipo `NaT`, não se consegue fazer a alteração do _timezone_. 

Nossa saída então é filtrar essa ocorrências e apagá-las.

Para nossa sorte, o Pandas nos dá uma colher de chá que já conhecemos: o `dropna`! 

In [ ]:
df.dropna(inplace=True)

Para confirmar se houve a limpeza, vamos fazer uma inspeção em `date`.

In [ ]:
df['date'].unique()

E nada de `NaT`. Assim,  

In [ ]:
df['date'] = df['date'].apply(lambda x: ref_timezone(x))

In [ ]:
df['date']

e não temos mais nenhum erro!

Para facilitar ainda mais nossa vida, nós podemos fracionar essa informação disponível em `date`. Vamos fazer isso criando algumas novas variáveis.

A primeira variável será `daysofweek`, na qual colocaremos o dia da semana (segunda, terça, etc.). Para isso, usaremos o método `day_name` do próprio Pandas

In [ ]:
df['dayofweek'] = df['date'].apply(lambda x: x.day_name())

In [ ]:
df['dayofweek']

e depois vamos transformar a variável `dayofweek` em categórica, para facilitar futuras análises.

In [ ]:
df['dayofweek'] = pd.Categorical(df['dayofweek'], categories=[
    'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday',
    'Saturday', 'Sunday'], ordered=True)

In [ ]:
df.info()

Em seguida, criaremos `timeofday`, que apresentará a hora do dia, usando, para isso

In [ ]:
df['timeofday'] = df['date'].apply(lambda x: x.hour + x.minute/60 + x.second/3600)

a variável `year`, que trará o ano do email

In [ ]:
df['year'] = df['date'].apply(lambda x: x.year)

e a variável `month`, que trará o mês do email

In [ ]:
df['month'] = df['date'].apply(lambda x: x.month_name())

e que também transformaremos em categórica

In [ ]:
df['month'] = pd.Categorical(df['month'], categories=[
    'January', 'February', 'March', 'April', 'May', 'June', 'July', "August", 
    'September', 'October', 'November', 'December'], ordered=True)

E como separamos todas as informações referentes a data em partes que possam facilitar a análise, não faz mais sentido manter a coluna `date`. Assim,

In [ ]:
df.drop('date', axis=1, inplace=True)

In [ ]:
df.info()

Por fim, minha sugestão é gravar esse dataframe pré-processado em um arquivo .csv novo. Por que fazer isso? Porque se for necessário fechar o notebook por algum motivo, não será necessário fazer novamente todo o processo que fizemos até aqui.

Para fazer isso, usaremos

In [ ]:
df.to_csv('emails_processadoss.csv')

In [ ]:
df